
<h1 align="center">Cell Tracking according to Biological Needs</h1>
<h2 align="center">Strong Mitosis-aware Random-finite Sets Tracker with Aleatoric Uncertainty</h2>

<p align="center">
  <img alt="Teaser Image" src="https://github.com/Dionakra/BiologicalNeeds/blob/main/assets/framework.png?raw=1">
  <br>
    <em>The output distribution of EmbedTrack using different input transformations on BF-C2DL-HSC.</em>
</p>

# Select the Dataset you want to process

In [1]:
# Select the dataset you want to process
DATASET = 'DIC-C2DH-HeLa'
SUBSET = 'train'
SEQUENCE = '01'

# Select some Inference Parameter
SHIFTS = 50

# Check if the selection is valid
AVAILABLE_DATASETS = [
    "Fluo-N2DH-SIM+",
    "Fluo-C2DL-MSC",
    "Fluo-N2DH-GOWT1",
    "PhC-C2DL-PSC",
    "BF-C2DL-HSC",
    "Fluo-N2DL-HeLa",
    "BF-C2DL-MuSC",
    "DIC-C2DH-HeLa",
    "PhC-C2DH-U373",
]
AVAILABLE_SUBSETS = ['train', 'challenge']
AVAILABLE_SEQUENCES = ['01', '02']
assert DATASET in AVAILABLE_DATASETS, f"Invalid dataset: {DATASET}"
assert SUBSET in AVAILABLE_SUBSETS, f"Invalid subset: {SUBSET}"
assert SEQUENCE in AVAILABLE_SEQUENCES, f"Invalid sequence: {SEQUENCE}"

# Setting up the environment

Creating the environment
  - downloading the CTC data
  - downloading the pretrained models.

In [2]:
!git clone https://github.com/Dionakra/BiologicalNeeds
!mv BiologicalNeeds/* .

# Download the data
if SUBSET == 'train':
    !python utils/prepare_data.py --datasets $DATASET --train
else:
    !python utils/prepare_data.py --datasets $DATASET --challenge

Cloning into 'BiologicalNeeds'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (367/367), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 367 (delta 229), reused 310 (delta 180), pack-reused 0 (from 0)
Receiving objects: 100% (367/367), 1.42 MiB | 36.34 MiB/s, done.
Resolving deltas: 100% (229/229), done.
mv: 'BiologicalNeeds' and './BiologicalNeeds' are the same file
python3: can't open file '/content/utils/prepare_data.py': [Errno 2] No such file or directory


In [3]:
# Download the pretrained models
import zipfile

!curl -O https://www.tnt.uni-hannover.de/de/project/MPT/data/BiologicalNeeds/models.zip
with zipfile.ZipFile("models.zip") as z:
    z.extractall("")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  236M  100  236M    0     0  15.4M      0  0:00:15  0:00:15 --:--:-- 17.6M


!git clone https://github.com/Dionakra/BiologicalNeeds

fatal: destination path 'BiologicalNeeds' already exists and is not an empty directory.
mv: cannot move 'BiologicalNeeds/models' to './models': Directory not empty


# Inference

## 1) EmbedTrack with Aleatoric Uncertainty by Advanced Test-Time Augmentation

In [ ]:
import os
from embedtrack.scripts.inference import process

res_path = os.path.join(os.getcwd(), 'results/embedtrack')

process(
    model_dir=os.path.join(os.getcwd(), 'models'),
    res_dir=res_path,
    shifts=[0, SHIFTS],
    multiscale=False,
    multi_segmentation=False,
    batch_size=32,
    refine_segmentation=False,
    raw_data_paths=[os.path.join(os.getcwd(), "Data", "train")],
    datasets=[DATASET],
    sequences=[SEQUENCE],
)

## 2) Multi Hypothesis Tracking

In [ ]:
from mht.scripts.inference import infer_all

in_path = res_path
res_path = os.path.join(os.getcwd(), 'results/mht')

infer_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
    tracker_args={},
)

## 3) Interpolation to fill temporal gaps in trajectories

In [ ]:
from utils.interpolate import postprocess_all

in_path = res_path
res_path = os.path.join(os.getcwd(), 'results/interpolated')

postprocess_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)

## 4) Postprocessing to satisfy the CTC requirements

In [ ]:
from utils.postprocess import postprocess_all

postprocess_all(
    data_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)